# Call Agent Analysis Experiment

This notebook runs the Call Agent Analysis Experiment using multiple LLM configurations and writes results to BigQuery.

In [ ]:
# MES - 02_run_call_agent_analysis_experiment.py
import os
import sys
import pandas as pd

project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from agents.call_agent_analysis_agent import CallAgentAnalysisAgent
from orchestration.experiment_runner import ExperimentRunner
from orchestration.bigquery_writer import BigQueryWriter
from core.gcp_client import (
    get_gcs_audio_bucket_name,
    get_gcs_audio_dataset_paths,
    get_bq_analysis_table_id,
    config as main_global_config
)

def run_agent_analysis_experiment():
    print("--- Starting Call Agent Analysis Experiment (Multi-LLM) ---")

    runner = ExperimentRunner(agent_class=CallAgentAnalysisAgent)
    print("Initialized ExperimentRunner.")

    gcs_bucket = get_gcs_audio_bucket_name()
    dataset_paths = get_gcs_audio_dataset_paths()
    dataset_key = 'sample_analysis_set'
    audio_folder_path = dataset_paths.get(dataset_key)

    if not audio_folder_path or 'your-gcs-bucket' in gcs_bucket or 'path/to' in audio_folder_path:
        print(f"ERROR: Please configure GCS bucket and '{dataset_key}' path in configs/main_config.yaml.")
        return
    print(f"Targeting GCS: gs://{gcs_bucket}/{audio_folder_path}")

    all_defined_llm_configs = list(main_global_config['gcp']['vertex_ai']['llm_configurations'].keys())
    if not all_defined_llm_configs:
        print("ERROR: No LLM configurations found in 'configs/main_config.yaml'.")
        return
        
    # llm_configs_to_test = all_defined_llm_configs # Test all
    llm_configs_to_test = ["gemini_1_5_flash_creative"] # Example: test only one specific config
    llm_configs_to_test = [name for name in llm_configs_to_test if name in all_defined_llm_configs]
    if not llm_configs_to_test:
        print(f"ERROR: None of the specified llm_configs_to_test were found. Available: {all_defined_llm_configs}")
        return
    print(f"LLM configurations to be tested: {llm_configs_to_test}")

    global_llm_params_override = {"max_output_tokens": 600}
    print(f"Global LLM parameter overrides: {global_llm_params_override}")
    
    # global_prompt_override = "Analyze this agent's call for quality: {transcript}"
    global_prompt_override = None
    if global_prompt_override: print(f"Global prompt override: '{global_prompt_override}'")

    print(f"\nStarting experiment...")
    results = runner.run_experiment(
        gcs_audio_folder_path=audio_folder_path,
        gcs_bucket_name=gcs_bucket,
        llm_config_names=llm_configs_to_test,
        global_llm_parameters_override=global_llm_params_override,
        global_prompt_override=global_prompt_override
    )

    if not results:
        print("Experiment finished but produced no results.")
        return
    print(f"\nExperiment completed. Generated {len(results)} result entries.")
    results_df = pd.DataFrame(results)
    print("\nFirst 2 experiment result entries (if any):")
    print(results_df.head(2).to_string())

    try:
        bq_writer = BigQueryWriter()
        analysis_table_id = get_bq_analysis_table_id()
        print(f"\nAttempting to write {len(results)} results to BigQuery table: {bq_writer.dataset_id}.{analysis_table_id}")
        bq_writer.write_results(results, analysis_table_id)
        print("Successfully wrote results to BigQuery.")
    except Exception as e:
        print(f"Error writing results to BigQuery: {e}")
        fallback_filename = "agent_analysis_experiment_results_fallback.csv"
        results_df.to_csv(fallback_filename, index=False)
        print(f"Results saved locally to {fallback_filename} due to BigQuery error.")

    print("--- Call Agent Analysis Experiment Finished ---")

if __name__ == "__main__":
    run_agent_analysis_experiment()